**Function Calling**

In [1]:
!pip install pandas mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 945.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [2]:
import pandas as pd

# Assuming we have the following data
data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

# Create DataFrame
df = pd.DataFrame(data)

**Step 1. User: specify tools and query**

Tools

In [3]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps({'status': df[df.transaction_id == transaction_id].payment_status.item()})
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps({'date': df[df.transaction_id == transaction_id].payment_date.item()})
    return json.dumps({'error': 'transaction id not found.'})

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Get payment status of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Get payment date of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    }
]

In [5]:
import functools

names_to_functions = {
  'retrieve_payment_status': functools.partial(retrieve_payment_status, df=df),
  'retrieve_payment_date': functools.partial(retrieve_payment_date, df=df)
}

**User Query**

In [8]:
from mistralai.models.chat_completion import ChatMessage

messages = [
    ChatMessage(role="user", content="What's the status of my transaction T1001?")
]

**Step 2. Model: Generate function arguments**

In [9]:
from mistralai.client import MistralClient

model = "mistral-small-latest"
api_key="use your API key" # I remove my API Key After execute this

client = MistralClient(api_key=api_key)

response = client.chat(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

response

ChatCompletionResponse(id='f531c9c684d34fa6bb1956af6f13a0a2', object='chat.completion', created=1721211797, model='mistral-small-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ltn2VBVbv', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))], tool_call_id=None), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=166, total_tokens=196, completion_tokens=30))

In [10]:
messages.append(response.choices[0].message)

In [11]:
messages

[ChatMessage(role='user', content="What's the status of my transaction T1001?", name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ltn2VBVbv', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))], tool_call_id=None)]

**Step 3. User: Execute function to obtain tool results**

In [12]:
import json
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  retrieve_payment_status 
function_params:  {'transaction_id': 'T1001'}


In [13]:
function_result = names_to_functions[function_name](**function_params)
function_result

'{"status": "Paid"}'

In [14]:
messages.append(ChatMessage(role="tool", name=function_name, content=function_result, tool_call_id=tool_call.id))

In [15]:
messages

[ChatMessage(role='user', content="What's the status of my transaction T1001?", name=None, tool_calls=None, tool_call_id=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='ltn2VBVbv', type=<ToolType.function: 'function'>, function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'))], tool_call_id=None),
 ChatMessage(role='tool', content='{"status": "Paid"}', name='retrieve_payment_status', tool_calls=None, tool_call_id='ltn2VBVbv')]

**Step 4. Model: Generate final answer**

In [16]:
response = client.chat(
    model=model,
    messages=messages
)

response.choices[0].message.content

'Your transaction T1001 has been paid successfully.'